# Lab 5: Text-to-Speech (TTS)


In this lab, first, we will study how to train a neural-based TTS model Tacotron 2 with  using SpeechBrain on the LJSpeech corpus.


#### <span style="color:green"> Text-to-Speech (TTS) with Tacotron2 trained on LJSpeech (with a pretrained model) </span>


The pre-trained model takes as input a short text and produces as output a spectrogram. One can get the final waveform by applying a vocoder (e.g., HiFIGAN) on top of the generated spectrogram.

#### Perform TTS with a pretrained model




In [1]:
import os
import torch
import torchaudio
import IPython
import matplotlib.pyplot as plt
from speechbrain.inference.TTS import Tacotron2
from speechbrain.inference.vocoders import HIFIGAN

torch.random.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

/home/pmagron/Enseignement/asr-tts-course/labs/.venv/lib/python3.12/site-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()


In [2]:
# Intialize TTS (tacotron2) and Vocoder (HiFIGAN)
tacotron2 = Tacotron2.from_hparams(source="speechbrain/tts-tacotron2-ljspeech", savedir="tmpdir_tts")
hifi_gan = HIFIGAN.from_hparams(source="speechbrain/tts-hifigan-ljspeech", savedir="tmpdir_vocoder")

# Running the TTS
mel_output, mel_length, alignment = tacotron2.encode_text("Hello!")

# Running Vocoder (spectrogram-to-waveform)
waveforms = hifi_gan.decode_batch(mel_output)

# Save the waverform
torchaudio.save('example_TTS.wav', waveforms.squeeze(1), 22050)

IPython.display.Audio(data=waveforms.squeeze(1), rate=22050)

hyperparams.yaml: 0.00B [00:00, ?B/s]

/home/pmagron/Enseignement/asr-tts-course/labs/.venv/lib/python3.12/site-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()


model.ckpt:   0%|          | 0.00/113M [00:00<?, ?B/s]

hyperparams.yaml: 0.00B [00:00, ?B/s]

/home/pmagron/Enseignement/asr-tts-course/labs/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


generator.ckpt:   0%|          | 0.00/55.8M [00:00<?, ?B/s]

/home/pmagron/Enseignement/asr-tts-course/labs/.venv/lib/python3.12/site-packages/torchaudio/_backend/utils.py:337: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.save_with_torchcodec` under the hood. Some parameters like format, encoding, bits_per_sample, buffer_size, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's encoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.encoders.AudioEncoder
  warnings.warn(
/home/pmagron/Enseignement/asr-tts-course/labs/.venv/lib/python3.12/site-packages/torchaudio/_backend/ffmpeg.py:247: UserWarning: torio.io._streaming_media_encoder.StreamingMediaEncoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/au

#### <span style="color:green"> Train Text-to-Speech (TTS) </span>

The model is trained with SpeechBrain (on a small subset of the origonal corpus)

Complete 3 steps:


1. ##### Reduce number of files in the train dataset:

In [3]:
! mv LJSpeech-1.1/metadata.csv LJSpeech-1.1/metadata_original.csv
! head -n 128 LJSpeech-1.1/metadata_original.csv > LJSpeech-1.1/metadata.csv

mv: impossible d'évaluer 'LJSpeech-1.1/metadata.csv': Aucun fichier ou dossier de ce nom
/bin/bash: ligne 1: LJSpeech-1.1/metadata.csv: Aucun fichier ou dossier de ce nom


2. ##### Change params in train.yaml:

    Path: speechbrain/recipes/LJSpeech/TTS/tacotron2/hparams/train.yaml

        - epochs: 8
        - keep_checkpoint_interval: 1
        - batch_size: 8


3. ##### Run training:

In [4]:
! python \
  speechbrain/recipes/LJSpeech/TTS/tacotron2/train.py \
  --device=cpu \
  --max_grad_norm=1.0 \
  --data_folder=LJSpeech-1.1 \
  speechbrain/recipes/LJSpeech/TTS/tacotron2/hparams/train.yaml

python: can't open file '/home/pmagron/Enseignement/asr-tts-course/labs/lab4-5/speechbrain/recipes/LJSpeech/TTS/tacotron2/train.py': [Errno 2] No such file or directory


<span style="color:red"> **Exercise:**</span>
<span style="color:orange"> **Speech-to-text training**</span>

The training of the full model is too long, and in this excercise,  we will try only several epochs of traing on the subset of the traing corpus in order to speed up the experiment:

1. Train 8 (or more) epochs: 
 - keep_checkpoint_interval: 1
 - batch_size: 8
 - 128 files of the LJSpeech

2. Report logs with loss values on train and validation dataset. Which losses were used in the training?

3. Do you observe overfitting? Please explain your answer.